In [1]:
import cv2
import torch
import torch.nn as nn
import numpy as np
from torchvision import models, transforms
from PIL import Image

# Setup Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 Running on: {device}")

# GTSRB Label Mapping (The 43 Classes)
GTSRB_CLASSES = {
    0: 'Speed Limit 20 km/h', 1: 'Speed Limit 30 km/h', 2: 'Speed Limit 50 km/h', 
    3: 'Speed Limit 60 km/h', 4: 'Speed Limit 70 km/h', 5: 'Speed Limit 80 km/h', 
    6: 'End 80 km/h Limit', 7: 'Speed Limit 100 km/h', 8: 'Speed Limit 120 km/h', 
    9: 'No Passing', 10: 'No Truck Passing', 11: 'Priority Crossroad', 
    12: 'Priority Road', 13: 'Yield', 14: 'Stop', 15: 'No Vehicles', 
    16: 'No Trucks', 17: 'No Entry', 18: 'Caution', 19: 'Curve Left', 
    20: 'Curve Right', 21: 'Double Curve', 22: 'Bumpy Road', 23: 'Slippery Road', 
    24: 'Narrow Road', 25: 'Road Work', 26: 'Traffic Signals', 27: 'Pedestrians', 
    28: 'Children Crossing', 29: 'Bicycles Crossing', 30: 'Ice/Snow', 
    31: 'Wild Animals', 32: 'End Speed + Passing', 33: 'Turn Right Ahead', 
    34: 'Turn Left Ahead', 35: 'Ahead Only', 36: 'Go Straight or Right', 
    37: 'Go Straight or Left', 38: 'Keep Right', 39: 'Keep Left', 
    40: 'Roundabout Mandatory', 41: 'End No Passing', 42: 'End No Truck Passing'
}

🚀 Running on: cuda
